In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf

import math
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pickle
import gzip
import tensorflow as tf

In [2]:
import keras
keras.__version__

'2.4.3'

# Data Load

In [9]:
import random  
seed_num = 42
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4068), (6182,), (1545, 10, 4068), (1545,))

In [10]:
def MAE(y_true, y_pred):
    return "{:.2e}".format(np.mean(np.abs(y_true-y_pred)))

def MSE(y_true, y_pred):
    return "{:.2e}".format(np.mean(np.square(y_true-y_pred)))

def RMSE(y_true, y_pred):
    return "{:.2e}".format(np.sqrt(np.mean(np.square(y_true-y_pred))))

In [11]:
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.ensemble import VotingClassifier
from keras.layers import InputLayer, GRU

# Modeling

In [15]:
# # 2. LSTM model
# def lstm_model():
#     seed_num = 42 # model_seed
#     tf.random.set_seed(seed_num)

#     lstm = Sequential()
#     lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
#     lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
#     lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
#     lstm.add(Dropout(0.2))
#     lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
#     lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
#     lstm.add(Dropout(0.2))
#     lstm.add(Dense(units=1, activation='sigmoid'))
#     return lstm

# def sorted_ls(path):
#     mtime=lambda f: os.stat(os.path.join(path, f)).st_mtime
#     return list(sorted(os.listdir(path), key=mtime))

# path = '/project/LSH/'
# with tf.device('/device:GPU:0'):
#     # 1. Data load
#     x = np.load(path + 'x_(7727,10,4068).npy')
#     y = np.load(path + 'y_(7727,1).npy')

#     # 3-1. Best model saving
#     MODEL_SAVE_FOLDER_PATH = './models/'
#     if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
#         os.mkdir(MODEL_SAVE_FOLDER_PATH)

#     acc_list, precision_list, recall_list, f1_list, auc_list = [], [], [], [], []
#     sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)

#     # 4. Crossvalidation
#     for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
#         X_train, y_train = x[train_index,:,:], y[train_index]
#         X_test, y_test = x[test_index,:,:], y[test_index]

#         # 3-2. Best model saving
#         model_path = MODEL_SAVE_FOLDER_PATH + f'/seed={seed}_'+'{epoch:02d}'+'_val_loss:{val_loss:.4f}.hdf5'

#         cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
#                                         verbose=1, save_best_only=True)
#         early_stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1, restore_best_weights=False)


#         model = lstm_model()
#         model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
#                       loss = "binary_crossentropy", metrics=['acc'])
#         model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,
#                   callbacks=[early_stop, cb_checkpoint], shuffle=False)


In [12]:
tf.random.set_seed(seed_num)

In [ ]:
%%time
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                      loss = "binary_crossentropy", metrics=['acc'])
lstm.summary()

estimator = []
for i in range(1,11):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:lstm, epochs=20, batch_size=256)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
print("RMSE : ", RMSE(y_test, preds))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 10, 128)           2148864   
_________________________________________________________________
lstm_13 (LSTM)               (None, 10, 64)            49408     
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 10, 64)            33024     
_________________________________________________________________
lstm_15 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

25/25 [==============================] - 2s 63ms/step - loss: 0.0731 - acc: 0.9840
Epoch 17/20
25/25 [==============================] - 2s 63ms/step - loss: 0.0730 - acc: 0.9820
Epoch 18/20
25/25 [==============================] - 2s 65ms/step - loss: 0.0865 - acc: 0.9778
Epoch 19/20
25/25 [==============================] - 2s 64ms/step - loss: 0.0782 - acc: 0.9817
Epoch 20/20
25/25 [==============================] - 2s 64ms/step - loss: 0.0677 - acc: 0.9848
Epoch 1/20
25/25 [==============================] - 2s 69ms/step - loss: 0.0696 - acc: 0.9846
Epoch 2/20
25/25 [==============================] - 2s 69ms/step - loss: 0.0781 - acc: 0.9808
Epoch 3/20
25/25 [==============================] - 2s 68ms/step - loss: 0.0754 - acc: 0.9819
Epoch 4/20
25/25 [==============================] - 2s 71ms/step - loss: 0.0672 - acc: 0.9856
Epoch 5/20
25/25 [==============================] - 2s 70ms/step - loss: 0.0667 - acc: 0.9854
Epoch 6/20
25/25 [==============================] - 2s 70ms/step - 

- range(1,2) : 정확도 : 0.7469255663430421, RMSE :  5.03e-01
- range(1,11) :